In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()
using TrajOptPlots
using StaticArrays
using Altro
using TrajectoryOptimization

 Activating environment at `~/.julia/dev/TrajOptPlots/notebooks/Project.toml`


## Launch Visualizer
We start by launching a `MeshCat` visualizer. We can use `render` to open the visualizer in the most relevant format for our current context (renders in a cell for a Jupyter Notebook).

In [2]:
using MeshCat
vis = Visualizer()
render(vis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8700
└ @ MeshCat /home/brian/.julia/dev/MeshCat/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()), Set{Any}(), ip"127.0.0.1", 8700))

## Visualizing a Model
Let's start with one of the models that already has it's visualization defined. All of the models in [`RobotZoo.jl`](https://github.com/RoboticExplorationLab/RobotZoo.jl) have visualization methods defined in `TrajOptPlots`. Let's pick the canonical cartpole.

To add the model to our visualizer, we use the `TrajOptPlots.set_mesh!` method.

In [3]:
# Load the Cartpole model
using RobotZoo: Cartpole

# Instantiate the cartpole model
model = Cartpole()

# Add model to the visualizer
TrajOptPlots.set_mesh!(vis, model)

MeshCat Visualizer with path /meshcat/robot/cart/pole at http://127.0.0.1:8700

### Changing the color
Sometimes we want to change the color of the model, for example, if we want to plot multiple copies. All of the models in `RobotZoo` allow at least one color to be changed via the `color` argument:

In [4]:
using Colors   # import the colorant string
# Change the color
TrajOptPlots.set_mesh!(vis, model, color=colorant"blue")

MeshCat Visualizer with path /meshcat/robot/cart/pole at http://127.0.0.1:8700

### Adding another model
The model, by default is added under the `robot` path in the MeshCat visualizer tree. We can add another model by simply using a different path in the tree. We don't see it in the visualizer since it's directly on top of the existing model.

In [5]:
# add another robot
TrajOptPlots.set_mesh!(vis["robot_copy"], model)

MeshCat Visualizer with path /meshcat/robot_copy/robot/cart/pole at http://127.0.0.1:8700

### Deleting a Model or Scene
If you want to delete a model, simply use the `delete!` method provided by MeshCat on the `robot` in the tree:

In [6]:
delete!(vis["robot"]);             # deletes the blue model at the root of the vis tree.

In [7]:
delete!(vis["robot_copy/robot"]);  # deletes the other model we added

In [8]:
delete!(vis);                      # clears the scene

## Visualizing a State
We can change the model configuration with the `visualize!` method. Note that we have to pass it a `StaticVector`.

In [9]:
# Change the robot state
TrajOptPlots.set_mesh!(vis, model)   # add the model back
x = SA[0.5, pi/4, 0, 0]              # [x,θ,xdot,θdot]  (velocities don't matter)
visualize!(vis, model, x)

MeshCat Visualizer with path /meshcat/robot/cart/pole/geom at http://127.0.0.1:8700

### Visualizing a Trajectory
We can visualize an entire trajectory using the same `visualize!` method, this time passing in either a vector of states and a final time, or an `AbstractTrajectory`.

In [10]:
# Solve the cartpole problem
prob, opts = Altro.Problems.Cartpole()
solver = ALTROSolver(prob, opts)
solve!(solver)

# Get the trajectory
X = states(solver)
tf = prob.tf

# Visualize with a vector of states and final time
visualize!(vis, model, tf, X)

# Visualize with a trajectory
Z = get_trajectory(solver)
visualize!(vis, model, Z)

# Visualize with the problem or solver
visualize!(vis, prob)
visualize!(vis, solver)

## Visualizing multiple trajectories
We can compare different results by visualizing multiple trajectories at the same time.

In [11]:
# Change initial condition and re-solve
prob2 = copy(Problem(prob, x0=SA[+0.5, 0, 0.1, 0]))
prob3 = copy(Problem(prob, x0=SA[-0.5, 0, 0.1, 0]))
solver2 = ALTROSolver(prob2, opts)
solver3 = ALTROSolver(prob3, opts)
solve!(solver2)
solve!(solver3)
vis = Visualizer()
TrajOptPlots.set_mesh!(vis, prob.model)
render(vis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8701
└ @ MeshCat /home/brian/.julia/dev/MeshCat/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("robot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("rod" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x72, 0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x72, 0x6f, 0x64], nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()),"cart" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("pole" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x2f, 0x63, 0x61, 0x72, 0x74, 0x2f, 0x70, 0x6f, 0x6c, 0x65], Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("cyl" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x65, 0x2f, 0x67, 0x65, 0x6f, 0x6d, 0x2f, 0x63, 0x79, 0x6c], nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()),"mass" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x67, 0x65, 0x6f, 0x6d, 0x2f, 0x6d, 0x61, 0x73, 0x73], nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()))))),"box" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x74, 0x2f, 0x63, 0x61, 0x72, 0x74, 0x2f, 0x62, 0x6f, 0x78], nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()),"hinge" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x63, 0x61, 0x72, 0x74, 0x2f, 0x68, 0x69, 0x6e, 0x67, 0x65], nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()))))))))), Set{Any}(), ip"127.0.0.1", 8701))

In [12]:
# Visualize both trajectories
visualize!(vis, solver, solver2)
visualize!(vis, model, get_trajectory(solver), get_trajectory(solver2))
visualize!(vis, model, tf, states(solver), states(solver2))

### Changing the color
Using the `color` argument for the `set_mesh!` method, `visualize!` allows us a way to change the color of the models to distinguish between them. Passing `nothing` as a color will use the default colors.

In [13]:
# Make copies a single color
visualize!(vis, solver, solver2, solver3, colors=colorant"blue")

In [14]:
# Set individual colors
visualize!(vis, solver, solver2, solver3, colors=[colorant"blue", nothing, colorant"purple"])

## Waypoints
It's often useful to visualize a set of waypoints, for example, for an image in a publication. TrajOptPlots provides a convenient method similar to the `visualize!` method. Be careful when the model is a large mesh, since this method may take a significant amount of time (e.g. the YakPlane in RobotZoo).

In [15]:
# clear the robot copies
delete!(vis["robot_copies"])

# visualize waypoints evenly distributed in time
waypoints!(vis, solver2, length=11);

In [16]:
# visualize waypoints at specific time indices
waypoints!(vis, solver2, inds=[1,10,20,30,40,50,60,70,75,80,85,90,95,101]);

### Changing the color
Similar to the `visualize!` method we can change the color. We can leverage the `range` method defined on colors. Using `HSL` colors tends to provide more vibrant colors.

In [17]:
waypoints!(vis, solver2, length=21, 
    color=colorant"green", 
    color_end=colorant"red"
)

MeshCat Visualizer with path /meshcat/robot/cart/pole/geom at http://127.0.0.1:8701

In [18]:
waypoints!(vis, solver2, length=21, 
    color=HSL(colorant"green"), 
    color_end=HSL(colorant"red")
)

MeshCat Visualizer with path /meshcat/robot/cart/pole/geom at http://127.0.0.1:8701

In [19]:
# delete the waypoints
TrajOptPlots.clear_waypoints!(vis)

# delete robot copies
TrajOptPlots.clear_copies!(vis)

MeshCat Visualizer with path /meshcat/robot_copies at http://127.0.0.1:8701

## Custom Models
If you define your own model, you will need to define how you want TrajOptPlots to visualize it. This is done via two separate methods. First, we need to define the geometry of the model, which we do using `TrajOptPlots._set_mesh!`

In [20]:
using RobotDynamics
using GeometryBasics
using CoordinateTransformations
using Rotations
struct DoubleCartpole{T} <: AbstractModel
    l1::T
    l2::T
end
state_dim(::DoubleCartpole) = 6
control_dim(::DoubleCartpole) = 1
# we won't define the dynamics here

function TrajOptPlots._set_mesh!(vis, model::DoubleCartpole; color=nothing)
    # Define the geometry
    dim = Vec(0.1, 0.4, 0.1)  # size of the cart
    rod = Cylinder(Point3f0(0,-10,0), Point3f0(0,10,0), 0.03f0)
    cart = Rect3D(-dim/2, dim)
    hinge1 = Cylinder(
        Point3f0(-dim[1]/2, dim[2]/4, dim[3]/2), 
        Point3f0(dim[1], dim[2]/4, dim[3]/2), 
        0.03f0
    )
    hinge2 = Cylinder(
        Point3f0(-dim[1]/2, -dim[2]/4, dim[3]/2), 
        Point3f0(dim[1], -dim[2]/4, dim[3]/2), 
        0.03f0
    )
    color = isnothing(color) ? colorant"green" : color
    pole1 = Cylinder(Point3f0(0,0,0), Point3f0(0,0,model.l1), 0.01f0)
    pole2 = Cylinder(Point3f0(0,0,0), Point3f0(0,0,model.l2), 0.01f0)
    mass1 = Sphere(Point3f0(0,0,model.l1), 0.05f0)
    mass2 = Sphere(Point3f0(0,0,model.l2), 0.05f0)
    
    # Place geometry in visualizer tree and define the material properties
    setobject!(vis["rod"], rod, MeshPhongMaterial(color=colorant"grey"))
    setobject!(vis["cart","box"], cart, MeshPhongMaterial(color=color))
    setobject!(vis["cart","hinge1"], hinge1, MeshPhongMaterial(color=colorant"black"))
    setobject!(vis["cart","hinge2"], hinge2, MeshPhongMaterial(color=colorant"black"))
    setobject!(vis["cart","pole1","geom","cyl"], pole1, MeshPhongMaterial(color=colorant"grey"))
    setobject!(vis["cart","pole2","geom","cyl"], pole2, MeshPhongMaterial(color=colorant"grey"))
    setobject!(vis["cart","pole1","geom","mass"], mass1, MeshPhongMaterial(color=color))
    setobject!(vis["cart","pole2","geom","mass"], mass2, MeshPhongMaterial(color=color))
    
    # Move the poles to the hinged
    settransform!(vis["cart","pole1"], Translation(0.75dim[1], dim[2]/4, dim[3]/2))
    settransform!(vis["cart","pole2"], Translation(0.75dim[1], -dim[2]/4, dim[3]/2))
end

vis = Visualizer()
TrajOptPlots.set_mesh!(vis, DoubleCartpole(.5,.5), color=colorant"firebrick")
render(vis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8703
└ @ MeshCat /home/brian/.julia/dev/MeshCat/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("robot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("rod" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x72, 0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x72, 0x6f, 0x64], nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()),"cart" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("hinge2" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x61, 0x72, 0x74, 0x2f, 0x68, 0x69, 0x6e, 0x67, 0x65, 0x32], nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()),"pole1" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x63, 0x61, 0x72, 0x74, 0x2f, 0x70, 0x6f, 0x6c, 0x65, 0x31], Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("cyl" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x31, 0x2f, 0x67, 0x65, 0x6f, 0x6d, 0x2f, 0x63, 0x79, 0x6c], nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()),"mass" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x67, 0x65, 0x6f, 0x6d, 0x2f, 0x6d, 0x61, 0x73, 0x73], nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()))))),"hinge1" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x61, 0x72, 0x74, 0x2f, 0x68, 0x69, 0x6e, 0x67, 0x65, 0x31], nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()),"box" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x74, 0x2f, 0x63, 0x61, 0x72, 0x74, 0x2f, 0x62, 0x6f, 0x78], nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()),"pole2" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x63, 0x61, 0x72, 0x74, 0x2f, 0x70, 0x6f, 0x6c, 0x65, 0x32], Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("cyl" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x32, 0x2f, 0x67, 0x65, 0x6f, 0x6d, 0x2f, 0x63, 0x79, 0x6c], nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()),"mass" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x67, 0x65, 0x6f, 0x6d, 0x2f, 0x6d, 0x61, 0x73, 0x73], nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()))))))))))))), Set{Any}(), ip"127.0.0.1", 8703))

After defining what the geometry looks like, we need to define how it changes with the state. We define our state vector to be `[x, theta1, theta2, xdot, theta1dot, theta2dot]`. We just need to define our custom `visualize!` method:

In [21]:
function TrajOptPlots.visualize!(vis, model::DoubleCartpole, x::SVector)
    y = x[1]
    θ1,θ2 = x[2], x[3]
    q1 = expm(SA[1,0,0]*(θ1-pi))
    q2 = expm(SA[1,0,0]*(θ2-pi))    
    settransform!(vis["robot","cart"], Translation(0,y,0))
    settransform!(vis["robot","cart","pole1","geom"], LinearMap(q1))
    settransform!(vis["robot","cart","pole2","geom"], LinearMap(q2))
end

# Test it out 
visualize!(vis, DoubleCartpole(.5,.5), SA[1,deg2rad(0),deg2rad(180)]);

In [22]:
# Define a new state vector and visualize a trajectory
X = [SA[x[1],x[2],x[2],0,0,0] for x in states(solver)]
visualize!(vis, DoubleCartpole(0.5,0.5), prob.tf, X)

## Rigid Bodies
Many robotic systems can be modeled as a single rigid body moving freely in three-dimensional space. These systems are easy to visualize, so TrajOptPlots provides some convenient methods for working with these systems. 

In [23]:
# Load the Quadrotor model
using RobotZoo: Quadrotor
vis = Visualizer()
TrajOptPlots.set_mesh!(vis, Quadrotor())
render(vis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8704
└ @ MeshCat /home/brian/.julia/dev/MeshCat/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("robot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x72, 0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()))))))), Set{Any}(), ip"127.0.0.1", 8704))

To visualize these systems we can provide the `RobotDynamics.RBState` type, which assumes the rotation is modeled as quaternion, and provides some useful methods for converting between different rotation parameterizations. For instance, if our model uses Modifed Rodrigues Parameters (MRPs), we can visualize states using any rotation convention:

In [24]:
model = Quadrotor{MRP}()
visualize!(vis, model, RBState(zeros(3), rand(MRP), zeros(3), zeros(3)))
visualize!(vis, model, RBState(zeros(3), RotX(pi/6), zeros(3), zeros(3)))
visualize!(vis, model, RBState(zeros(3), expm(SA[0,0,1]*pi/4), zeros(3), zeros(3)));

If we have a trajectory of `RBState`s we can visualize it as follows. Note that MeshCat
automatically interpolates between the states.

In [25]:
X = [rand(RBState) for x = 1:10]
X2 = [RobotDynamics.build_state(model, x) for x in X]
visualize!(vis, model, tf, X2, colors=[colorant"blue"])

## Default visualization
If a custom `visualize!` method isn't defined for a model, it will treat it as a rigid body and try to convert the state to a `RBState` using `RBState(model, x)`. One easy way to visualize is to simply define this method instead of the `visualize!` method. This method should be preferred for any model that inherits from `RobotDynamics.RigidBody`.